In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import scipy
from scipy.stats import beta
from scipy.stats import binomtest

path_to_data =  #Input path to data
data = pd.read_csv(path_to_data)


def iv_test(prob_mat):
    n   = prob_mat.shape[1]
    iv_sums = np.zeros((2,n)) #0-same, 1-diff
    for j in range(n):
        iv_sums[0,j] = prob_mat[0,j,0] + prob_mat[1,j,1]
        iv_sums[1,j] = prob_mat[1,j,0] + prob_mat[0,j,1]
    return iv_sums

def output_count(samples):
    '''
    Takes samples as input and outputs the fraction of the samples that
    satisfy the IV inequality. Calls iv_test by passing it the conditional
    distribution based on the samples.
    '''
    count = 0
    for i in samples:
        jnt = np.reshape(i,(2,6,2))
        cnd = jnt/np.sum(np.sum(jnt,axis=2,keepdims=True),axis=1,keepdims=True) #gXdXa, summing over d and a to get conditional on s
        iv_sum_res = iv_test(cnd)
        if np.sum(iv_sum_res>=1)==0:#satisfy IV
            count = count +1
        else:
            continue
    c = count/samples.shape[0]
    return c

## The procedure

In [ ]:
#Sample from prior distribution 
sup_gen = (data["Gender"].unique()).shape[0]
sup_dom = (data["Dept"].unique()).shape[0] #n
sup_out = (data["Admit"].unique()).shape[0]

alpha = np.ones(sup_gen*sup_dom*sup_out) #Parameter for the Uniform Dirichlet prior
prior_sample_size = int(1e6)
posterior_sample_size = int(1e6)

prior_samples = np.random.dirichlet(alpha, prior_sample_size) #Sampling from prior
c = output_count(prior_samples) #fraction of prior samples that satisfy the IV inequality
#sample counts
sample_counts = np.zeros((sup_gen,sup_dom,sup_out))
gender_counter = 0
#The way the data is entered, the resulting samples are of the shape (2,n,2) where the first axis corresponds to 
#gender, the second to the department and the third to the outcome. 
for gender in data["Gender"].unique():
    dept_counter=0
    for dept in data["Dept"].unique():
        sample_counts[gender_counter,dept_counter,0] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0]
        sample_counts[gender_counter,dept_counter,1] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0]
        dept_counter=dept_counter+1
    gender_counter=gender_counter+1
#Updated sample counts that are used to update the posterior 
posterior_samples = np.random.dirichlet(alpha+np.reshape(sample_counts,(alpha.shape)),posterior_sample_size) #Sampling from posterior
f = output_count(posterior_samples) #fraction of posterior samples that satisfy the IV inequality
num_success = int(f*posterior_sample_size)
print('The confidence interval of the posterior probability is [' + str(binomtest(num_success, posterior_sample_size).proportion_ci()[0]) + ',' +str(binomtest(num_success,posterior_sample_size).proportion_ci()[1])+']')

## Trying different priors

In [ ]:
sup_gen = (data["Gender"].unique()).shape[0]
sup_dom = (data["Dept"].unique()).shape[0] #n
sup_out = (data["Admit"].unique()).shape[0]
alp=0.05 

prior_sample_size = int(1e6)
posterior_sample_size = int(1e6)
f_vec = []
#Sampling from prior

scale_vec  = np.logspace(np.log10(0.01),5,20)
for scale in scale_vec:
    alpha = scale*np.ones(sup_gen*sup_dom*sup_out) #Parameter for the Uniform Dirichlet prior
    prior_samples = np.random.dirichlet(alpha, prior_sample_size) 
    c = output_count(prior_samples) #fraction of prior samples that satisfy the IV inequality
    print('prior done')
    #sample counts
    sample_counts = np.zeros((sup_gen,sup_dom,sup_out))
    gender_counter = 0
    #The way the data is entered, the resulting samples are of the shape (2,n,2) where the first axis corresponds to 
    #gender, the second to the department and the third to the outcome. 
    for gender in data["Gender"].unique():
        dept_counter=0
        for dept in data["Dept"].unique():
            sample_counts[gender_counter,dept_counter,0] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0]
            sample_counts[gender_counter,dept_counter,1] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0]
            dept_counter=dept_counter+1
        gender_counter=gender_counter+1
    #Updated sample counts that are used to update the posterior 
    posterior_samples = np.random.dirichlet(alpha+np.reshape(sample_counts,(alpha.shape)),posterior_sample_size) #Sampling from posterior
    f = output_count(posterior_samples) #fraction of posterior samples that satisfy the IV inequality
    num_successf*posterior_sample_size)
    f_vec.append(binomtest(num_success, posterior_sample_size).proportion_ci()[0])

plt.semilogx(scale_vec,np.array(f_vec),marker='o')
plt.xlabel('alpha')
plt.ylabel('Lower limit of confidence interval')
plt.show()

## Frequentist test

In [ ]:
#Implementing the IV inequality test that Wang, Robins, Richardson 2017 propose 
from scipy.stats import chisquare

sup_gen = (data["Gender"].unique()).shape[0]
sup_dom = (data["Dept"].unique()).shape[0] #n
sup_out = (data["Admit"].unique()).shape[0]

#Construct variable (Q) from data
sample_counts = np.zeros((sup_gen,sup_dom,sup_out))
table_counts  = np.zeros((sup_dom,sup_out,4))
gender_counter = 0
for gender in data["Gender"].unique():
    dept_counter=0
    for dept in data["Dept"].unique():
        sample_counts[gender_counter,dept_counter,0] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0]
        sample_counts[gender_counter,dept_counter,1] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0]
        if gender_counter == 0:
            table_counts[dept_counter,0,0] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0] #Q^d0=0 for Z=0
            table_counts[dept_counter,0,1] = sum(data.loc[(data.Gender==gender)].Freq) - data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0] #Q^d0=1 for Z=0
            table_counts[dept_counter,1,0] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0] #Q^d1=0 for Z=0
            table_counts[dept_counter,1,1] = sum(data.loc[(data.Gender==gender)].Freq) - data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0]#Q^d1=1 for Z=0
        if gender_counter == 1:
            table_counts[dept_counter,0,3] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0] #Q^d0=1 for Z=1
            table_counts[dept_counter,0,2] = sum(data.loc[(data.Gender==gender)].Freq) - data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Admitted')].Freq.iloc[0] #Q^d0=0 for Z=1
            table_counts[dept_counter,1,3] = data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0] #Q^d1=1 for Z=1
            table_counts[dept_counter,1,2] = sum(data.loc[(data.Gender==gender)].Freq) - data.loc[(data.Gender==gender)&(data.Dept==dept)&(data.Admit=='Rejected')].Freq.iloc[0]#Q^d1=0 for Z=1
        dept_counter=dept_counter+1
    gender_counter=gender_counter+1

for d in range(sup_dom):
    for a in range(sup_out):
        c_00 = table_counts[d,a,3]
        c_01 = table_counts[d,a,2]
        c_10 = table_counts[d,a,1]
        c_11 = table_counts[d,a,0]
        N = c_00 + c_01 + c_10 + c_11
        e_00 = N*((c_00+c_01)/N)*((c_00 + c_10)/N)
        e_01 = N*((c_00+c_01)/N)*((c_01 + c_11)/N)
        e_10 = N*((c_00+c_10)/N)*((c_11 + c_10)/N)
        e_11 = N*((c_11+c_10)/N)*((c_11 + c_01)/N)
        obs = [c_00,c_01,c_10,c_11]
        exp = [e_00,e_01,e_10,e_11]
        print(c_00/(c_00+c_01) - c_10/(c_10+c_11))
        print(chisquare(obs,exp)[1])